
1-face Detection

In [3]:
import cv2

# Load the pre-trained Haar cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [4]:
image = cv2.imread(r"C:\Users\hp\id2\image1.jpg")

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [3]:
# Detect faces in the grayscale image
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

# Draw rectangles around the detected faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the result
cv2.imshow('Face Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

adjust the parameters

In [6]:
faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.05, minNeighbors=3, minSize=(20, 20))

# Draw rectangles around the detected faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the result
cv2.imshow('Face Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Using MTCNN (Multi-task Cascaded Convolutional Networks):


In [ ]:
pip install mtcnn


In [7]:
from mtcnn import MTCNN

# Load the image
image = cv2.imread(r"C:\Users\hp\id2\image1.jpg")

# Convert the image to RGB (MTCNN works with RGB images)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Initialize the MTCNN detector
detector = MTCNN()

# Detect faces in the image
faces = detector.detect_faces(image_rgb)

# Draw rectangles around detected faces
for face in faces:
    x, y, w, h = face['box']
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

# Display the result
cv2.imshow('Face Detection with MTCNN', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


MTCNN is better

# Face recognition Siamese networks

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model


In [10]:
from tensorflow.keras.layers import Subtract, Lambda

def create_siamese_model(input_shape):
    input_1 = Input(input_shape)
    input_2 = Input(input_shape)
    
    # Shared convolutional layers
    conv_net = Conv2D(32, (3, 3), activation='relu')
    x1 = conv_net(input_1)
    x2 = conv_net(input_2)
    
    x1 = MaxPooling2D()(x1)
    x2 = MaxPooling2D()(x2)
    
    x1 = Flatten()(x1)
    x2 = Flatten()(x2)
    
    # Calculate L1 distance between the encoded features
    l1_distance = Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([x1, x2])
    
    # Output layer
    output_layer = Dense(1, activation='sigmoid')(l1_distance)
    
    # Create model
    siamese_model = Model(inputs=[input_1, input_2], outputs=output_layer)
    return siamese_model


In [21]:
import os
import random
import shutil

fake_dataset_path = r"C:\Users\hp\Desktop\Codsoft_AI\real_and_fake_face\training_fake"
true_dataset_path = r"C:\Users\hp\Desktop\Codsoft_AI\real_and_fake_face\training_real"

# Create directories to store selected images
selected_fake_dir = "selected_fake"
selected_true_dir = "selected_true"
os.makedirs(selected_fake_dir, exist_ok=True)
os.makedirs(selected_true_dir, exist_ok=True)

# Function to randomly select images from a dataset
def select_images(dataset_path, num_images, output_dir):
    dataset_images = os.listdir(dataset_path)
    selected_images = random.sample(dataset_images, num_images)
    # Copy selected images to the output directory
    for image_name in selected_images:
        src_path = os.path.join(dataset_path, image_name
        dst_path = os.path.join(output_dir, image_name)
        shutil.copy(src_path, dst_path)

# Select 100 images from each dataset
select_images(fake_dataset_path, 100, selected_fake_dir)
select_images(true_dataset_path, 100, selected_true_dir)



In [30]:

fake_dir = r"C:\Users\hp\Desktop\Codsoft_AI\selected_fake"
real_dir = r"C:\Users\hp\Desktop\Codsoft_AI\selected_true"


In [36]:
import os

# Check if images exist in the directories
def check_images_exist(directory):
    try:
        num_images = len(os.listdir(directory))
        if num_images > 0:
            print(f"Images exist in the {directory} directory.")
            return True
        else:
            print(f"No images found in the {directory} directory.")
            return False
    except FileNotFoundError:
        print(f"The directory {directory} does not exist.")
        return False

In [39]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images_from_directory(directory, target_size=(100, 100)):
    image_paths = [os.path.join(directory, filename) for filename in os.listdir(directory)]
    images = []
    for image_path in image_paths:
        image = load_img(image_path, target_size=target_size)
        image_array = img_to_array(image) / 255.0  # Normalize pixel values
        images.append(image_array)
    return np.array(images)

# Load images from fake and real directories
fake_images = load_images_from_directory(fake_dir)
real_images = load_images_from_directory(real_dir)

# Check if images were loaded successfully
if len(fake_images) > 0 and len(real_images) > 0:
    print("Images loaded successfully!")
    print(f"Number of fake images: {len(fake_images)}")
    print(f"Number of real images: {len(real_images)}")
else:
    print("Failed to load images. Please check the directories.")


Images loaded successfully!
Number of fake images: 100
Number of real images: 100


In [42]:
import numpy as np

# Define labels for fake and real images
fake_labels = np.zeros(len(fake_images))  # Assign label 0 for fake images
real_labels = np.ones(len(real_images))   # Assign label 1 for real images

# Concatenate images and labels
X = np.concatenate([fake_images, real_images], axis=0)
y = np.concatenate([fake_labels, real_labels], axis=0)

# Shuffle the data
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
X = X[indices]
y = y[indices]

# Split the data into training and validation sets
split = int(0.8 * len(X))  # 80% for training, 20% for validation
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# Print the shapes of the training and validation sets
print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation data shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)


Training data shape: (160, 100, 100, 3)
Training labels shape: (160,)
Validation data shape: (40, 100, 100, 3)
Validation labels shape: (40,)


In [52]:
# Check the shape of the original input data
print("Shape of X_train[:, 0]:", X_train[:, 0].shape)
print("Shape of X_train[:, 1]:", X_train[:, 1].shape)




Shape of X_train[:, 0]: (160, 100, 3)
Shape of X_train[:, 1]: (160, 100, 3)


In [47]:
# Instantiate the Siamese model
siamese_model = create_siamese_model(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Check the summary of the model
siamese_model.summary()



Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 100, 100,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ input_layer_12      │ (None, 100, 100,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 98, 98,    │     896 │ input_layer_11[0][0… │
│                     │ 32)               │         │ input_layer_12[0][0] │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_14    │ (None, 49, 49,    │       0 │ conv2d_16[0][0]      │
│ (MaxPooling2D)      │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ max_pooling2d_15    │ (None, 49, 49,    │       0 │ conv2d_16[1][0]      │
│ (MaxPooling2D)      │ 32)               │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_10          │ (None, 76832)     │       0 │ max_pooling2d_14[0]… │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ flatten_11          │ (None, 76832)     │       0 │ max_pooling2d_15[0]… │
│ (Flatten)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_3 (Lambda)   │ (None, 76832)     │       0 │ flatten_10[0][0],    │
│                     │                   │         │ flatten_11[0][0]     │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_10 (Dense)    │ (None, 1)         │  76,833 │ lambda_3[0][0]       │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 77,729 (303.63 KB)

 Trainable params: 77,729 (303.63 KB)

 Non-trainable params: 0 (0.00 B)

In [55]:
import cv2
import numpy as np

def load_and_preprocess_images(image_paths, target_size=(100, 100)):
    images = []
    for path in image_paths:
        # Load the image using OpenCV
        image = cv2.imread(path)
        # Resize the image to the target size
        image = cv2.resize(image, target_size)
        # Convert the image to RGB format
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Append the preprocessed image to the list
        images.append(image)
    # Convert the list of images to a NumPy array
    images = np.array(images)
    return images

# Example usage:
real_image_paths = [r"C:\Users\hp\Desktop\Codsoft_AI\selected_true\real_00123.jpg"]

In [59]:
X_real = load_and_preprocess_images(real_image_paths)
print("Shape of X_real:", X_real.shape)


Shape of X_real: (1, 100, 100, 3)
